# Chapter 35: Slots and Weak References

This notebook explores two advanced memory-optimization techniques in Python: `__slots__` for reducing per-instance memory overhead, and `weakref` for creating references that do not prevent garbage collection. Together, these patterns are essential for building high-performance, memory-efficient systems.

## Key Concepts
- **`__slots__`**: Replaces per-instance `__dict__` with a fixed set of attribute slots
- **Memory savings**: Slotted objects use significantly less memory than regular objects
- **`weakref.ref`**: Creates a reference that does not keep the object alive
- **`WeakValueDictionary`**: A dictionary whose values are weak references, auto-cleaned on garbage collection
- **Caching patterns**: Using weak references for memory-friendly caches

## Section 1: `__slots__` Basics

By default, Python stores instance attributes in a per-instance `__dict__`. Defining `__slots__` in a class replaces the dictionary with a fixed-size structure, restricting which attributes can be set.

In [ ]:
class PointRegular:
    """A regular class with __dict__."""

    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y


class PointSlotted:
    """A slotted class with fixed attributes."""

    __slots__ = ("x", "y")

    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y


# Both work the same for basic access
r = PointRegular(1.0, 2.0)
s = PointSlotted(3.0, 4.0)

print(f"Regular: ({r.x}, {r.y})")
print(f"Slotted: ({s.x}, {s.y})")

# Regular has __dict__, slotted does not
print(f"\nRegular has __dict__: {hasattr(r, '__dict__')}")
print(f"Slotted has __dict__: {hasattr(s, '__dict__')}")

print(f"\nRegular __dict__: {r.__dict__}")

## Section 2: `__slots__` Restricts Attributes

With `__slots__`, you can only set attributes that are listed in the slots tuple. Attempting to set an unlisted attribute raises `AttributeError`.

In [ ]:
class Point:
    __slots__ = ("x", "y")


p = Point()
p.x = 1
p.y = 2
print(f"Point: ({p.x}, {p.y})")

# Trying to set an unlisted attribute fails
try:
    p.z = 3  # type: ignore[attr-defined]
    print("This should not print")
except AttributeError as e:
    print(f"\nAttributeError: {e}")

# Listing the available slots
print(f"\nSlots: {Point.__slots__}")

## Section 3: Memory Savings with `__slots__`

The primary benefit of `__slots__` is memory reduction. Each instance no longer carries a `__dict__` (which is a hash table). The savings are significant when creating many instances.

In [ ]:
import sys


class RegularPoint:
    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y


class SlottedPoint:
    __slots__ = ("x", "y")

    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y


r = RegularPoint(1.0, 2.0)
s = SlottedPoint(1.0, 2.0)

# sys.getsizeof measures the object itself (not its __dict__)
r_size: int = sys.getsizeof(r) + sys.getsizeof(r.__dict__)
s_size: int = sys.getsizeof(s)

print(f"Regular point size:  {r_size} bytes (object + __dict__)")
print(f"Slotted point size:  {s_size} bytes")
print(f"Savings per object:  {r_size - s_size} bytes ({(1 - s_size / r_size) * 100:.0f}%)")

# With 100,000 objects the savings add up
n: int = 100_000
print(f"\nWith {n:,} objects:")
print(f"  Regular: ~{r_size * n / 1024 / 1024:.1f} MB")
print(f"  Slotted: ~{s_size * n / 1024 / 1024:.1f} MB")

## Section 4: `__slots__` with Inheritance

When using `__slots__` with inheritance, each class in the hierarchy should declare only its own new slots. If a parent does not use `__slots__`, the subclass will still have a `__dict__`.

In [ ]:
class Base:
    __slots__ = ("x",)


class Child(Base):
    __slots__ = ("y",)  # Only new slots, not 'x' again


c = Child()
c.x = 1
c.y = 2
print(f"Child: x={c.x}, y={c.y}")
print(f"Has __dict__: {hasattr(c, '__dict__')}")


# If a subclass does NOT define __slots__, it gets a __dict__
class ChildWithDict(Base):
    pass  # No __slots__ defined


d = ChildWithDict()
d.x = 1
d.z = 99  # This works because __dict__ is available
print(f"\nChildWithDict: x={d.x}, z={d.z}")
print(f"Has __dict__: {hasattr(d, '__dict__')}")
print(f"__dict__: {d.__dict__}")

## Section 5: Combining `__slots__` with `__dict__`

You can include `'__dict__'` in `__slots__` to get both fixed slots and a dynamic dictionary. This gives you the best of both worlds: fast access for common attributes and flexibility for extras.

In [ ]:
class Hybrid:
    __slots__ = ("x", "y", "__dict__")

    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y


h = Hybrid(1.0, 2.0)

# Slotted attributes
print(f"x={h.x}, y={h.y}")

# Dynamic attributes via __dict__
h.color = "red"  # type: ignore[attr-defined]
h.label = "origin"  # type: ignore[attr-defined]
print(f"color={h.color}, label={h.label}")

# x and y are NOT in __dict__ (they use slots)
print(f"\n__dict__: {h.__dict__}")
print(f"'x' in __dict__: {'x' in h.__dict__}")
print(f"'color' in __dict__: {'color' in h.__dict__}")

## Section 6: Weak References with `weakref.ref`

A weak reference does not increase an object's reference count, so it does not prevent garbage collection. When the referent is collected, the weak reference returns `None`.

In [ ]:
import weakref


class MyObj:
    """A simple class that supports weak references."""

    def __init__(self, name: str) -> None:
        self.name = name

    def __repr__(self) -> str:
        return f"MyObj({self.name!r})"


# Create an object and a weak reference to it
obj = MyObj("example")
ref: weakref.ref[MyObj] = weakref.ref(obj)

# The weak reference is callable -- calling it returns the referent
print(f"ref() = {ref()}")
print(f"ref() is obj: {ref() is obj}")

# Delete the strong reference
del obj

# Now the weak reference returns None
print(f"\nAfter del obj:")
print(f"ref() = {ref()}")

## Section 7: Weak Reference Callbacks

You can pass a callback to `weakref.ref` that is called when the referent is about to be garbage-collected.

In [ ]:
import weakref


class Resource:
    def __init__(self, name: str) -> None:
        self.name = name


def cleanup_callback(ref: weakref.ref) -> None:
    """Called when the referent is about to be collected."""
    print(f"  Callback: object was garbage collected (ref={ref})")


# Create object with callback on weak reference
r = Resource("database_connection")
ref: weakref.ref[Resource] = weakref.ref(r, cleanup_callback)

print(f"Before del: ref() = {ref()!r}")
print(f"Name: {ref().name}")

# Deleting the strong reference triggers garbage collection
print("\nDeleting strong reference...")
del r
print(f"After del: ref() = {ref()}")

## Section 8: `WeakValueDictionary`

A `WeakValueDictionary` holds weak references as its values. When a value object is garbage collected, its entry is automatically removed from the dictionary. This is ideal for caches.

In [ ]:
import weakref


class Item:
    def __init__(self, name: str) -> None:
        self.name = name

    def __repr__(self) -> str:
        return f"Item({self.name!r})"


# Create a WeakValueDictionary
cache: weakref.WeakValueDictionary[str, Item] = weakref.WeakValueDictionary()

# Add items to the cache
item_a = Item("alpha")
item_b = Item("beta")
item_c = Item("gamma")

cache["a"] = item_a
cache["b"] = item_b
cache["c"] = item_c

print(f"Cache keys: {list(cache.keys())}")
print(f"Cache['a']: {cache['a']}")

# Delete one strong reference
del item_b
print(f"\nAfter del item_b:")
print(f"Cache keys: {list(cache.keys())}")
print(f"'b' in cache: {'b' in cache}")

# Delete another
del item_a
print(f"\nAfter del item_a:")
print(f"Cache keys: {list(cache.keys())}")

## Section 9: Caching Pattern with Weak References

A common pattern uses `WeakValueDictionary` to cache expensive objects so they are reused while alive, but do not prevent garbage collection when no longer needed.

In [ ]:
import weakref


class ExpensiveObject:
    """Simulates an object that is expensive to create."""

    _cache: weakref.WeakValueDictionary[str, "ExpensiveObject"] = (
        weakref.WeakValueDictionary()
    )
    _creation_count: int = 0

    def __init__(self, key: str) -> None:
        self.key = key
        ExpensiveObject._creation_count += 1
        print(f"  Created ExpensiveObject({key!r}) [creation #{self._creation_count}]")

    @classmethod
    def get_or_create(cls, key: str) -> "ExpensiveObject":
        """Return cached instance if available, otherwise create new."""
        obj = cls._cache.get(key)
        if obj is not None:
            print(f"  Cache hit for {key!r}")
            return obj
        print(f"  Cache miss for {key!r}")
        obj = cls(key)
        cls._cache[key] = obj
        return obj


# First access creates the object
obj1 = ExpensiveObject.get_or_create("config")

# Second access returns the cached object
obj2 = ExpensiveObject.get_or_create("config")
print(f"\nSame object: {obj1 is obj2}")

# Delete strong references -- object can be collected
del obj1, obj2

# Next access must create a new one
print()
obj3 = ExpensiveObject.get_or_create("config")
print(f"Total creations: {ExpensiveObject._creation_count}")

## Section 10: `__slots__` and `__weakref__`

By default, slotted classes do not support weak references. To enable them, include `'__weakref__'` in `__slots__`.

In [ ]:
import weakref


class SlottedNoWeakref:
    __slots__ = ("value",)


class SlottedWithWeakref:
    __slots__ = ("value", "__weakref__")


# Cannot create weak reference to SlottedNoWeakref
obj1 = SlottedNoWeakref()
try:
    ref1 = weakref.ref(obj1)
except TypeError as e:
    print(f"Cannot weakref SlottedNoWeakref: {e}")

# Can create weak reference to SlottedWithWeakref
obj2 = SlottedWithWeakref()
obj2.value = 42
ref2: weakref.ref[SlottedWithWeakref] = weakref.ref(obj2)
print(f"\nWeakref works: ref2().value = {ref2().value}")

del obj2
print(f"After del: ref2() = {ref2()}")

## Section 11: `WeakKeyDictionary`

While `WeakValueDictionary` has weak values, `WeakKeyDictionary` has weak keys. Entries are removed when the key object is garbage collected.

In [ ]:
import weakref


class Session:
    def __init__(self, user: str) -> None:
        self.user = user

    def __repr__(self) -> str:
        return f"Session({self.user!r})"


# Track metadata about sessions without preventing their cleanup
session_metadata: weakref.WeakKeyDictionary[Session, dict[str, str]] = (
    weakref.WeakKeyDictionary()
)

s1 = Session("alice")
s2 = Session("bob")

session_metadata[s1] = {"ip": "10.0.0.1", "browser": "Firefox"}
session_metadata[s2] = {"ip": "10.0.0.2", "browser": "Chrome"}

print(f"Tracked sessions: {len(session_metadata)}")
print(f"Alice's metadata: {session_metadata[s1]}")

# When a session is deleted, its metadata is automatically cleaned up
del s1
print(f"\nAfter del s1: tracked sessions = {len(session_metadata)}")
print(f"Remaining keys: {list(session_metadata.keys())}")

## Summary

### `__slots__`
- **`__slots__ = ("attr1", "attr2")`**: Replaces per-instance `__dict__` with fixed attribute slots
- **Memory savings**: Significant when creating many instances (no per-object hash table)
- **Attribute restriction**: Only listed attributes can be set; others raise `AttributeError`
- **Inheritance**: Each class should declare only its own new slots
- **`__dict__` in slots**: Include `"__dict__"` in `__slots__` for hybrid behavior
- **`__weakref__` in slots**: Include `"__weakref__"` to support weak references

### `weakref`
- **`weakref.ref(obj)`**: Creates a weak reference; call it to get the referent or `None`
- **Callbacks**: `weakref.ref(obj, callback)` calls `callback` when the referent is collected
- **`WeakValueDictionary`**: Values are weak references; entries auto-removed on collection
- **`WeakKeyDictionary`**: Keys are weak references; entries auto-removed on collection

### Caching Patterns
- Use `WeakValueDictionary` for caches that do not prevent garbage collection
- `get_or_create` pattern: check cache first, create and cache on miss
- Combine `__slots__` with `__weakref__` for memory-efficient weakly-referenceable objects